In [1]:
#--build and train a basic CNN pytorch model for cifar10 dataset

In [2]:
#--load & understand dataset -> prepare tensors dataset -> prepare train/val -> prepare batches
#--model -> build model -> train model -> evaluate model
#--inference on sample images
#--save and load model
#--convert to onnx model and print model architecture

In [3]:
#---import drive to mount and load dataset
from google.colab import drive
drive.mount('/content/mount_gdrive')

Mounted at /content/mount_gdrive


In [4]:
#--import required libraries
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
from torch.utils.data import DataLoader


In [5]:
#--load and understand dataset
data_save_path="/content/mount_gdrive/MyDrive/Learning_AI_dataset/pytorch_dataset/dataset"

dataset=CIFAR10(root=data_save_path,
              download=True,
              transform=ToTensor()
              )


Files already downloaded and verified


In [6]:
#--checkdataset type
print(len(dataset))

50000


In [7]:
print(dataset[0])

(tensor([[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
         [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
         [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
         ...,
         [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
         [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
         [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],

        [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
         [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
         [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
         ...,
         [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
         [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
         [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],

        [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
         [0.0784, 0.0000, 0.0000,  ..., 0.2157, 0.1961, 0.2235],
         [0.0824, 0.0000, 0.0314,  ..., 0.1961, 0.1961, 0

In [8]:
print(dataset[0][0].shape)

torch.Size([3, 32, 32])


In [9]:
#--split into train and val
train_ds,val_ds=random_split(dataset,[40000,10000])

#--prepare train and val dataloader
batch_size=32
train_loader=DataLoader(train_ds,batch_size,shuffle=True)
val_loader=DataLoader(val_ds,batch_size,shuffle=True)


In [10]:
#--check shapes for loader
for images, labels in train_loader: #--> single batch
  print(images.shape)
  break

torch.Size([32, 3, 32, 32])


In [11]:
#--define accuracy function
def acc(preds,labels):
  max_prob,max_prob_indices=torch.max(preds,dim=1)
  return torch.tensor(torch.sum(max_prob_indices==labels).item()/len(preds))

In [12]:
#--build model helper functions

class cnn_helper(nn.Module):
  def training_step(self,batch): #--perform model training for each batch
    images,labels=batch
    preds=self(images)
    # preds=F.softmax(preds,dim=1)
    loss=F.cross_entropy(preds,labels)

    return loss

  def validation_step(self,batch): #--perform model validation/inference for each batch
    images,labels=batch
    preds=self(images)
    # preds=F.softmax(preds,dim=1)
    val_loss=F.cross_entropy(preds,labels)
    val_acc=acc(preds,labels)

    return {
        "val_loss":val_loss,
        "val_acc":val_acc
    }

  def validation_epoch_end(self,outputs): #--accumulate for entire dataset/epoch then compute average val loss and acc

    batch_loss=[x['val_loss'] for x in outputs]
    batch_acc=[x['val_acc'] for x in outputs]

    avg_batch_loss=torch.stack(batch_loss).mean()
    avg_batch_acc=torch.stack(batch_acc).mean()

    return {
         "val_loss":avg_batch_loss.item(),
        "val_acc":avg_batch_acc.item()
    }

  def epoch_end(self,epoch,result):
    print("Epoch : [{}], val_loss : [{:.4f}], val_acc : [{:.4f}]".format(epoch,result['val_loss'],result['val_acc']))


In [14]:
#--define model architecture:

class cifar10_cnn_model(cnn_helper):
  def __init__(self):
    super().__init__()
    self.network=nn.Sequential(
        #--1st block -> input [batch_size,3,32,32]
        nn.Conv2d(3,32,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),#--output [batch_size,16,16,16]

        #--2nd block -> input [batch_size,16,16,16]
        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),#--output [batch_size,32,8,8]

        #--3rd block -> input [batch_size,32,8,8]
        nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),#--output [batch_size,64,4,4]

        #--final block
        nn.Flatten(),
        nn.Linear(256*4*4,1024),
        nn.ReLU(),
        nn.Linear(1024,512),
        nn.ReLU(),
        nn.Linear(512,10)

    )

  def forward(self,xb):
    out=self.network(xb)
    return out

In [15]:
#--initiate model
model=cifar10_cnn_model()
model

cifar10_cnn_model(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=4096, out_features=1024, bias=True)
    (17): ReLU()
    (18): Linear(in_fe

In [16]:
# Print the shapes of weights and biases for each layer
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f'Layer: {name}, Shape: {param.shape} (Weight)')
    elif 'bias' in name:
        print(f'Layer: {name}, Shape: {param.shape} (Bias)')

Layer: network.0.weight, Shape: torch.Size([32, 3, 3, 3]) (Weight)
Layer: network.0.bias, Shape: torch.Size([32]) (Bias)
Layer: network.2.weight, Shape: torch.Size([64, 32, 3, 3]) (Weight)
Layer: network.2.bias, Shape: torch.Size([64]) (Bias)
Layer: network.5.weight, Shape: torch.Size([128, 64, 3, 3]) (Weight)
Layer: network.5.bias, Shape: torch.Size([128]) (Bias)
Layer: network.7.weight, Shape: torch.Size([128, 128, 3, 3]) (Weight)
Layer: network.7.bias, Shape: torch.Size([128]) (Bias)
Layer: network.10.weight, Shape: torch.Size([256, 128, 3, 3]) (Weight)
Layer: network.10.bias, Shape: torch.Size([256]) (Bias)
Layer: network.12.weight, Shape: torch.Size([256, 256, 3, 3]) (Weight)
Layer: network.12.bias, Shape: torch.Size([256]) (Bias)
Layer: network.16.weight, Shape: torch.Size([1024, 4096]) (Weight)
Layer: network.16.bias, Shape: torch.Size([1024]) (Bias)
Layer: network.18.weight, Shape: torch.Size([512, 1024]) (Weight)
Layer: network.18.bias, Shape: torch.Size([512]) (Bias)
Layer: n

In [27]:
#--start training process
epochs=10
opt=torch.optim.Adam
print(opt)

<class 'torch.optim.adam.Adam'>


In [37]:
@torch.no_grad() #--make sure no gradient computation during validation evaluation
def evaluate(model,val_loader):
  model.eval()
  outputs=[model.validation_step(batch) for batch in val_loader]#--iterrate through each batch and accumulate results
  return model.validation_epoch_end(outputs)

def fit(epochs,model,train_loader,val_loader,opt):
  #--iterrate through each epoch
  history=[]
  opt_fn=opt(model.parameters(),1e-3)

  for epoch in range(epochs):

    model.train()
    train_loss_list=[]
    train_acc_list=[]
    #--training phase -> iterrate through each batch
    for batch in train_loader:

      loss=model.training_step(batch)
      train_loss_list.append(loss)
      # print("train_loss:",loss)

      #--compute gradients
      loss.backward()
      #--update weights and bias
      opt_fn.step()
      #--reset the gradients
      opt_fn.zero_grad()


    #--validation phase -> once all batch completed for a given epoch
    results=evaluate(model,val_loader)
    model.epoch_end(epoch,results)
    history.append(results)

  return history


In [38]:
#--load data to gpu memory
def get_default_device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  else:
    return torch.device('cpu')

print(get_default_device())

#--
def to_device(data,device):
  if isinstance(data,(list,tuple)):
    return [to_device(x,device) for x in data]
  return data.to(device,non_blocking=True)

class DeviceDataLoader():
  def __init__(self,dl,device):
    self.dl=dl
    self.device=device

  def __iter__(self):
    for b in self.dl:
      yield to_device(b,self.device)

  def __len__(self):
    return len(self.dl)

cuda


In [39]:
device=get_default_device()
device

device(type='cuda')

In [40]:
train_loader1=DeviceDataLoader(train_loader,device)
val_loader1=DeviceDataLoader(val_loader,device)
to_device(model,device)

cifar10_cnn_model(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=4096, out_features=1024, bias=True)
    (17): ReLU()
    (18): Linear(in_fe

In [41]:
model=to_device(cifar10_cnn_model(),device)

In [42]:
evaluate(model,val_loader1)

{'val_loss': 2.303280830383301, 'val_acc': 0.10123801976442337}

In [43]:
# list(model.parameters())

In [45]:
#--start training
history=fit(epochs,model,train_loader1,val_loader1,opt)

Epoch : [0], val_loss : [0.9818], val_acc : [0.7213]
Epoch : [1], val_loss : [1.1560], val_acc : [0.7152]
Epoch : [2], val_loss : [1.1400], val_acc : [0.7217]
Epoch : [3], val_loss : [1.2126], val_acc : [0.7190]
Epoch : [4], val_loss : [1.2483], val_acc : [0.7174]
Epoch : [5], val_loss : [1.2352], val_acc : [0.7265]
Epoch : [6], val_loss : [1.4716], val_acc : [0.7204]
Epoch : [7], val_loss : [1.4714], val_acc : [0.7255]
Epoch : [8], val_loss : [1.5434], val_acc : [0.7218]
Epoch : [9], val_loss : [1.5278], val_acc : [0.7276]


In [46]:
torch.save(model.state_dict(),'basic_cnn_cifar10.pth')